In [4]:
#웹드라이버 매니저 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
ChromeDriverManager().install()

'/Users/mac/.wdm/drivers/chromedriver/mac64/126.0.6478.182/chromedriver-mac-arm64/chromedriver'

In [5]:
from selenium import webdriver
browser = webdriver.Chrome()

url= 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)


    


In [6]:

# 1페이지의 링크 데이터 전부 수집


#    https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
link_list = []
for i in range(1,4):
    print(f'현재 {i}페이지 수집중 입니다.')
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    ## 한개의 베스트 셀러 링크 수집
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지 전체의 링크데이터
    #browser.find_element(By.CLASS_NAME,'gd_name') #엘리먼트 : 요소
    datas = browser.find_elements(By.CLASS_NAME,'gd_name') #엘리먼트들: 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
            
    time.sleep(2)

print(link_list)

현재 1페이지 수집중 입니다.
현재 2페이지 수집중 입니다.
현재 3페이지 수집중 입니다.
['https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/123930880', 'https://www.yes24.com/Product/Goods/124225606', 'https://www.yes24.com/Product/Goods/127094902', 'https://www.yes24.com/Product/Goods/128133897', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/123930891', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/126511412', 'https://www.yes24.com/Product/Goods/126845471', 'https://www.yes24.com/Product/Goods/126702038', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/126516935', 'https://www.yes24.com/Product/Goods/126590468', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes2

In [8]:
len(link_list)

72

In [9]:
# 상세페이지로 이동
#for link in link_list:
#    browser.get(link)

browser.get(link_list[0])
title = browser.find_element(By.CLASS_NAME,'gd_name').text
author = browser.find_element(By.CLASS_NAME,'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME,'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME,'gd_date').text
rating = browser.find_element(By.CLASS_NAME,'yes_b').text
reviews = browser.find_element(By.CLASS_NAME,'txC_blue').text
sales = browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[0]
ranking_weeks = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[1]


In [15]:
#상세페이지 이동 후 데이터 크롤링
import pymysql      #pip3 install pymysql
import time
import re
from datetime import datetime

title = browser.find_element(By.CLASS_NAME,'gd_name').text
author = browser.find_element(By.CLASS_NAME,'gd_auth').text


publisher = browser.find_element(By.CLASS_NAME,'gd_pub').text
# 2020년 12월 25일 -> 2023-12-25 로 변경해줘야함
publishing = browser.find_element(By.CLASS_NAME,'gd_date').text
match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
if match:
    year, month, day = match.groups()
    data_obj = datetime(int(year),int(month),int(day))
    publishing = data_obj.strftime("%Y-%m-%d")
else:
    publishing = "9999-01-01"

rating = browser.find_element(By.CLASS_NAME,'yes_b').text
reviews = browser.find_element(By.CLASS_NAME,'txC_blue').text
reviews = int(reviews.replace(',',''))

sales = browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
sales = int(sales.replace(',',''))

price = browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
price = int(price.replace(',',''))


ranking = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[0].split(" ")[2].split('위')[0]
ranking_weeks = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[1].split(" ")[3][0]

print(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks)

하루 한 장 나의 어휘력을 위한 필사 노트 유선경 저 위즈덤하우스 2024-03-28 9.6 197 558822 23800 1 1


In [18]:

#데이터베이스 연동후 -> 수집한 데이터를 디비에 저장 (CSV)
import pymysql      #pip3 install pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect (
    host = 'localhost',
    user = 'root',
    password= '0000',
    db= 'yes24',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
    )


link_list = []
for i in range(1,4):
    print(f'현재 {i}페이지 수집중 입니다.')
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    ## 한개의 베스트 셀러 링크 수집
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지 전체의 링크데이터
    #browser.find_element(By.CLASS_NAME,'gd_name') #엘리먼트 : 요소
    datas = browser.find_elements(By.CLASS_NAME,'gd_name') #엘리먼트들: 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
    time.sleep(2)

    
with conn.cursor() as cur:
    
    for link in link_list:
        try:
            browser.get(link)
            
            title = browser.find_element(By.CLASS_NAME,'gd_name').text
            author = browser.find_element(By.CLASS_NAME,'gd_auth').text

            publisher = browser.find_element(By.CLASS_NAME,'gd_pub').text
            # 2020년 12월 25일 -> 2023-12-25 로 변경해줘야함
            publishing = browser.find_element(By.CLASS_NAME,'gd_date').text
            match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
            if match:
                year, month, day = match.groups()
                data_obj = datetime(int(year),int(month),int(day))
                publishing = data_obj.strftime("%Y-%m-%d")
            else:
                publishing = "9999-01-01"

            rating = browser.find_element(By.CLASS_NAME,'yes_b').text
            reviews = browser.find_element(By.CLASS_NAME,'txC_blue').text
            reviews = int(reviews.replace(',',''))

            sales = browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
            sales = int(sales.replace(',',''))

            price = browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
            price = int(price.replace(',',''))


            ranking = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[0].split(" ")[2].split('위')[0]
            ranking_weeks = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[1].split(" ")[3][0]
            print(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks)
            
            sql = """INSERT INTO BOOKS(
                title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks
                )
                VALUES(
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """
            cur.execute(sql,(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks))
            conn.commit()
            time.sleep(2)
        except Exception as e:
            print(e) 








RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods